In [1]:
import pandas as pd
import numpy as np
import polars as pl
import fastexcel

In [2]:
df1 = pl.read_csv("data/vic_mapping.csv")
df2 = pl.read_csv("data/mapping.csv")

In [8]:
df2.join(df1,how='left',on='Suburb/Town Name').write_csv("mapping_fnl.csv")

In [9]:
# Read in file
df = pl.read_excel("data/Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2025.xlsx", 
                   sheet_name="Table 03",
                #    read_options={
                #     "skip_rows": 6,
                #     }
                   )
# Column names change
# df.columns = ["GCCSA code",	"GCCSA name", "SA4 code", "SA4 name", "SA3 code",
#               "SA3 name", "SA2 code", "SA2 name","2023 Pop", "2024 Pop",
#               "2023-24 Change", "2023-24 Change %", "Natural Increase", "Net internal migration",
#               "Net oveseas migration","Area","Population Density"]
df_map = pl.read_csv("data/mapping_fnl.csv")

In [10]:
(
    df_map.filter(pl.col("Suburb/Town Name") == 'Templestowe')
)


Suburb/Town Name,Region,Local Government Area,Postcode
str,str,str,i64
"""Templestowe""","""Metro Inner East""",null,null


In [12]:
df_joined=(
    df
    .join(
        df_map,
        how='left',
        left_on='Suburb/Town Name',
        right_on='Suburb/Town Name'
    )
    .select(["Year", "Suburb/Town Name", "Offence Division", 'Offence Subdivision','Region','Incidents Recorded'])
    # .group_by(['Suburb/Town Name',
    #     'Offence Division', 'Offence Subdivision', 'Incidents Recorded 2024',
    # 'Incidents Recorded 2025','% change', '# change', 'Region'])
    # .filter(
    #     pl.col("Region").is_not_null()
    # )
    .with_columns(pl.col("Suburb/Town Name").str.replace("Melbourne", "Melbourne CBD"))
    .group_by(["Year","Region", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
    .agg(
        pl.sum("Incidents Recorded")

    )
    )
df_joined

Year,Region,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded
i64,str,str,str,str,i64
2016,"""Metro Outer East""","""Kilsyth""","""D Public order and security of…","""D20 Disorderly and offensive c…",5
2017,null,"""Colignan""","""E Justice procedures offences""","""E20 Breaches of orders""",2
2016,"""Other""","""Herne Hill""","""E Justice procedures offences""","""E20 Breaches of orders""",21
2025,null,"""Ellinbank""","""B Property and deception offen…","""B30 Burglary/Break and enter""",2
2025,null,"""Yarrambat""","""C Drug offences""","""C30 Drug use and possession""",1
…,…,…,…,…,…
2020,"""Metro Inner West""","""Ascot Vale""","""A Crimes against the person""","""A20 Assault and related offenc…",100
2020,"""Metro Inner East""","""Rowville""","""B Property and deception offen…","""B30 Burglary/Break and enter""",138
2019,"""Metro Inner North""","""Ivanhoe""","""B Property and deception offen…","""B30 Burglary/Break and enter""",148


In [13]:
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_tbl_rows(30)
df_2024=(
    df_joined
        .filter(pl.col("Year") == 2024)
        # .filter(pl.col("Offence Subdivision") == "B40 Theft")

        .group_by(["Year","Region", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )
        # .filter(pl.col("Local Government Area") == 'Darebin')
        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2024"})
        [:,1:7] # select relevant columns
)
df_2024.write_csv("data/df_crime_2024.csv")

In [14]:
pl.Config.set_tbl_rows(100)
df_2025=(
    df_joined
        .filter(pl.col("Year") == 2025)
        
        .group_by(["Year","Region", "Suburb/Town Name", "Offence Division", 'Offence Subdivision'])
        .agg(
            pl.sum("Incidents Recorded")
        )

        .sort("Incidents Recorded", descending=True)
        .rename({"Incidents Recorded" : "Incidents Recorded 2025"})
     
        [:,1:9] # select relevant columns
)
# df_2025.write_csv("data/df_crime_2025.csv")

In [15]:
pl.Config.set_tbl_rows(20)
df_crime=(
    df_2024
    .join(
        df_2025,
        on = ['Suburb/Town Name','Offence Subdivision'],
        how = 'left'
    )
    .select(["Region", "Suburb/Town Name", 
             "Offence Division", 'Offence Subdivision',"Incidents Recorded 2024",
             "Incidents Recorded 2025"])
    # .filter(
    #     pl.col("Local Government Area") == 'Darebin'
    # )
    .with_columns(
        (
        pl.when(pl.col("Incidents Recorded 2024")>10) # do not record pcp change if sample is under 10
        .then(
            ((pl.col("Incidents Recorded 2025")/pl.col("Incidents Recorded 2024")) - 1)*100)
        .otherwise(0)
        )
        .alias("% change")
    )
    .with_columns(
        (
        pl.col("Incidents Recorded 2025")-pl.col("Incidents Recorded 2024")
    ).alias("# change")
    )
    .sort("Incidents Recorded 2024", descending=True)
)

In [16]:
(
    df_crime
    .filter(
        pl.col('Suburb/Town Name') == 'Templestowe'
    )
)

Region,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded 2024,Incidents Recorded 2025,% change,# change
str,str,str,str,i64,i64,f64,i64
"""Metro Inner East""","""Templestowe""","""B Property and deception offences""","""B40 Theft""",133,133,0.0,0
"""Metro Inner East""","""Templestowe""","""B Property and deception offences""","""B30 Burglary/Break and enter""",107,90,-15.88785,-17
"""Metro Inner East""","""Templestowe""","""A Crimes against the person""","""A20 Assault and related offences""",34,31,-8.823529,-3
"""Metro Inner East""","""Templestowe""","""B Property and deception offences""","""B20 Property damage""",27,25,-7.407407,-2
"""Metro Inner East""","""Templestowe""","""B Property and deception offences""","""B50 Deception""",26,21,-19.230769,-5
"""Metro Inner East""","""Templestowe""","""E Justice procedures offences""","""E20 Breaches of orders""",24,36,50.0,12
"""Metro Inner East""","""Templestowe""","""A Crimes against the person""","""Other crimes against the person""",17,11,-35.294118,-6
"""Metro Inner East""","""Templestowe""","""C Drug offences""","""C30 Drug use and possession""",9,3,0.0,-6
"""Metro Inner East""","""Templestowe""","""A Crimes against the person""","""A70 Stalking, harassment and threatening behaviour""",9,7,0.0,-2


In [17]:
df_crime.write_csv("data/df_crime_suburb.csv")

In [99]:
df_crime.filter(pl.col("Region") == sel_region).group_by('Suburb/Town Name').agg(pl.sum("Incidents Recorded 2025")).sort(descending=False, by='Suburb/Town Name')

Suburb/Town Name,Incidents Recorded 2025
str,i64
"""Alphington""",420
"""Bellfield""",221
"""Brunswick""",2583
"""Brunswick East""",1033
"""Brunswick West""",970
"""Bundoora""",2320
"""Coburg""",2257
"""Coburg North""",1018
"""Fawkner""",889


In [84]:
df_crime.filter(pl.col("Suburb/Town Name") == selected_category).select(['Region']).head(1).item()

'Metro Inner North'

In [7]:
df_map = pl.read_csv("data/vic_mapping.csv")
df_crime = pl.read_csv("data/df_crime_suburb.csv")

In [16]:
df_crime.columns

['Region',
 'Suburb/Town Name',
 'Offence Division',
 'Offence Subdivision',
 'Incidents Recorded 2024',
 'Incidents Recorded 2025',
 '% change',
 '# change']

In [54]:
sel_suburb = 'Bulleen'
sel_region = 'Metro Inner East'

df_crime_summary_stg=(
    df_crime
        .filter(pl.col('Suburb/Town Name')==sel_suburb)
        .with_columns(pl.col("Offence Division").str.slice(2))
        .group_by(['Offence Division'])
        .agg([
            pl.sum("Incidents Recorded 2025").alias(sel_suburb + ' Incidents')
            ,pl.sum("# change").alias(sel_suburb + ' Incident Change')
        ]
        )
        .sort(by='Offence Division',descending=False)
).join(
        (
        df_crime
            .filter(pl.col('Region')==sel_region)
            .with_columns(pl.col("Offence Division").str.slice(2))
            .group_by(['Offence Division'])
            .agg([
                pl.mean("Incidents Recorded 2025").alias(sel_region + ' Incidents Avg')
                ,pl.mean("# change").alias(sel_region + ' Incident Avg Change')
            ]
            )
            .with_columns(pl.col(sel_region + ' Incidents Avg').cast(pl.Int64))
            .with_columns(pl.col(sel_region + ' Incident Avg Change').cast(pl.Int64))
            .sort(by='Offence Division',descending=False)
    ),
    on='Offence Division',
    how='left'
)

df_crime_summary_stg.select([df_crime_summary_stg.columns[0],df_crime_summary_stg.columns[1],
                  df_crime_summary_stg.columns[3],df_crime_summary_stg.columns[2],
                  df_crime_summary_stg.columns[4]])

Offence Division,Bulleen Incidents,Metro Inner East Incidents Avg,Bulleen Incident Change,Metro Inner East Incident Avg Change
str,i64,i64,i64,i64
"""Crimes against the person""",58,21,-14,1
"""Drug offences""",7,10,3,0
"""Justice procedures offences""",25,29,-17,-7
"""Property and deception offence…",341,125,104,30
"""Public order and security offe…",4,8,1,0


In [ ]:
(
    df_crime
        .filter(pl.col('Region')==sel_region)
        .with_columns(pl.col("Offence Division").str.slice(2))
        .group_by(['Offence Division'])
        .agg([
            pl.mean("Incidents Recorded 2025").alias(sel_region + ' Incidents Avg')
            ,pl.mean("# change").alias(sel_region + ' Incident Avg Change')
        ]
        )
        .with_columns(pl.col(sel_region + ' Incidents Avg').round(1))
        .with_columns(pl.col(sel_region + ' Incident Avg Change').round(1))
        .sort(by='Offence Division',descending=False)
)

Offence Division,Metro Inner East Incidents Avg,Metro Inner East Incident Avg Change
str,f64,f64
"""Crimes against the person""",21.7,1.7
"""Drug offences""",10.4,0.4
"""Justice procedures offences""",29.2,-7.8
"""Other offences""",1.1,-0.8
"""Property and deception offence…",125.5,30.4
"""Public order and security offe…",8.2,0.0


In [36]:
df_crime

Region,Suburb/Town Name,Offence Division,Offence Subdivision,Incidents Recorded 2024,Incidents Recorded 2025,% change,# change
str,str,str,str,i64,i64,f64,i64
"""Metro Inner""","""Melbourne CBD""","""B Property and deception offen…","""B40 Theft""",6062,9306,53.513692,3244
"""Metro Inner""","""Melbourne CBD""","""E Justice procedures offences""","""E20 Breaches of orders""",2202,1478,-32.879201,-724
"""Other""","""Dandenong""","""B Property and deception offen…","""B40 Theft""",1913,2865,49.764767,952
"""Metro Inner""","""Melbourne CBD""","""A Crimes against the person""","""A20 Assault and related offenc…",1816,2105,15.914097,289
"""Metro Inner North""","""Preston""","""B Property and deception offen…","""B40 Theft""",1767,3006,70.118846,1239
…,…,…,…,…,…,…,…
"""Metro Outer North""","""Donnybrook""","""C Drug offences""","""C20 Cultivate or manufacture d…",1,null,null,null
"""Other""","""Kennington""","""A Crimes against the person""","""A70 Stalking, harassment and t…",1,6,null,5
"""Other""","""Tatura""","""B Property and deception offen…","""B50 Deception""",1,4,null,3
